<a href="https://colab.research.google.com/github/genie0320/langchain/blob/main/prompt_240211.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set environment

In [ ]:
# !pip install -q -U langchain langchain-openai

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# colab 환경설정
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [ ]:
# !pip install -q icecream

In [ ]:
from icecream import ic
import pprint

# Set LLMs

In [ ]:
# !pip install -q -U langchain-openai

In [ ]:
from langchain_openai import OpenAI
compbot = OpenAI()

In [ ]:
from langchain_openai import ChatOpenAI

MODEL = 'gpt-3.5-turbo'
chatbot = ChatOpenAI(
    model_name = MODEL,
    temperature=0.7,
    max_tokens = 300
)

# Prompt

프롬프트에는 최소 2종류가 있는데,
- `PromptTemplate` : 일방적인 대답을 요구하는  
- `ChatPromptTemplate : `대화를 요구하는(그래서 나중에 히스토리로 쌓거나 할 때도 도움이 되는)

`from_template`로 틀을 잡고, `formate_prompt`로 값을 넣어주면 완성됨.
그리고 format된 결과물을 llm에 넘겨주면 대답을 해준다.

## Completion

In [ ]:
# from langchain_core.prompts import PromptTemplate, ChatPromptTemplate

# prompt_template = PromptTemplate.from_template('{subject}에 대한 재미있는 한문장짜리 농담을 해볼래?')
# prompt = prompt_template.format_prompt(subject='패션')

# res_comp = compbot.invoke(prompt)

In [ ]:
# ic(prompt_template) # PromptTemplate(input_variables=['subject'], template='{subject}에 대한 재미있는 한문장짜리 농담을 해볼래?')
# ic(prompt) # StringPromptValue(text='패션에 대한 재미있는 한문장짜리 농담을 해볼래?')
# prompt.to_string() # 패션에 대한 재미있는 한문장짜리 농담을 해볼래?
# ic(res_comp)

### 아래처럼 구성할 수도 있다. 사실 이쪽을 더 많이 쓴다.
template를 구성하면서 다양한 variable을 줄 수 있고, 이 변수값들은 여러가지 단계를 거치면서 채워질 수 있기 때문이다.(이를테면 문서검색결과, 웹검색결과, 사용자입력 등... )

In [ ]:
# template = '''
# 다음 주제에 대한 재미있는 한문장짜리 농담을 해볼래?

# <주제>
# {subject}
# '''

# prompt_template_02 = PromptTemplate(
#     input_variables=['subject'],
#     template = template
# )

# prompt_02 = prompt_template_02.format(subject = '패션')

# res_comp_02 = compbot.invoke(prompt_02)

In [ ]:
# ic(prompt_template_02)
# ic(prompt_02) # HumanMessage가 생성되는 것에 유의.
# prompt_02
# ic(res_comp_02) # AIMessage로 돌아오는 것에 유의.

In [ ]:
# chat_prompt_template = ChatPromptTemplate.from_template('{subject}에 대한 재미있는 한문장짜리 농담을 해볼래?')
# # chat_prompt = chat_prompt_template.format_messages(subject = '군고구마')
# chat_prompt = chat_prompt_template.format_prompt(subject = '군고구마')

# res_chat = chatbot.invoke(chat_prompt)

In [ ]:
# ic(chat_prompt_template)
# ic(chat_prompt) # HumanMessage가 생성되는 것에 유의.
# chat_prompt.to_string
# ic(res_chat) # AIMessage로 돌아오는 것에 유의.

## Chat

사용방법  
1. 전달할 template를 만들고  
`SystemMessagePromptTemplate.from_template(system_setting)`  
2. 그걸 prompt로 만들고    
`ChatPromptTemplate.from_messages([instruction, question])`  
3. 그 prompt에 query를 넣어서  
`chat_prompt = chat_prompt_template_02.format_prompt(subject = '군고구마').to_messages()`  
4. 발사.  
`invoke()`

잘못된 예 :  
- 최종 prompt는 format_prompt로 다듬어줘야 함.  
`chat_prompt = chat_prompt_template_02.format(subject = '군고구마')`  

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

# from langchain.schema import (
#     SystemMessage,
#     HumanMessage,
#     AIMessage
# )

In [ ]:
system_setting = '다음 주제에 대한 재미있는 한문장짜리 농담을 해볼래?'
instruction = SystemMessagePromptTemplate.from_template(system_setting)

# user_input = input('주제를 알려주세요.')
user_input = '{subject}'
question = HumanMessagePromptTemplate.from_template(user_input) # 사용자 입력부는, 가장 나중에 전달될 것이므로 '변수' 그 자체로 전달.

chat_prompt_template_02 = ChatPromptTemplate.from_messages([instruction, question]) # 위에서 각 만든 부품들을 조립해서 하나의 template로 만들고,..단 1개밖에 전달이 안되므로 배열로 감싸서 던진다.

chat_prompt_02 = chat_prompt_template_02.format_prompt(subject = '군고구마').to_messages() # 거기다가 마지막 사용자 질문을 입력해서 프롬프트를 완성.

# res_chat_03 = chatbot.invoke(chat_prompt_02)

In [ ]:
# ic(instruction)
# ic(question)
# ic(chat_prompt_template_02)
# ic(chat_prompt_02)
# ic(res_chat_03)
# print('\n\n', res_chat_03.content )

In [ ]:
import random

system_msg = SystemMessagePromptTemplate.from_template('사용자가 전달하는 숫자에 다음의 숫자를 더해줘.')
repeat = str(random.randrange(0,50))
user_input = '16'
human_msg = HumanMessagePromptTemplate.from_template(user_input)
prompt_full = ChatPromptTemplate.from_messages([system_msg, repeat, human_msg])

# res_full = chatbot.invoke(prompt_full.format_prompt())
# res_full

# Prompt Engineering

## Few-shot

백문이 불여일견이라는 말은 AI에게도 통하는 모양이다. 내가 원하는 결과물의 형태가 있는 경우, 구질구질 설명하지 말고 깔끔하게 샘플을 만들어서 보내주면 그걸 응용해서 결과물을 뽑아준다.

In [ ]:
examples = [
    {
      "question": "초등학교때는 어땠어?",
      "answer": """
        초등학교때 선생님은 너무 좋았어.
        하지만 애들이 나를 싫어했었어.
        그래서 별로 재미가 없었어.
        """,
    },
    {
      "question": "중학교때는 어땠어?",
      "answer": """
        중학교때 선생님은 좀 싫었어.
        하지만 애들이 나를 많이 좋아했었어.
        그래서 연애도 많이 했지.
        """,
    },
    {
        "question": "고등학교때는 어땠어?",
        "answer": """
        고등학교때 선생님은 예뻤어.
        애들도 많이 좋아했었어.
        그래서 나도 공부를 열심히 했지.
        """,
    }
]

In [ ]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate

prompt_template = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\nAnswer:{answer}"
)

# print(prompt_template.format(**examples[0]))

In [ ]:
few_shot_prompt = FewShotPromptTemplate( # 여기에서 비로소 FewShot~이 등장한다.
    examples=examples,
    example_prompt=prompt_template,
    suffix="Question: {input}", # suffix의 의미는... 'input값은 **이따가** 줄건데, 그건 Question 옆에다가 배치해줘...'
    input_variables=["input"],
)

print(few_shot_prompt.format(input="대학교때는 어땠어?"))

Question: 초등학교때는 어땠어?
Answer:
        초등학교때 선생님은 너무 좋았어. 
        하지만 애들이 나를 싫어했었어. 
        그래서 별로 재미가 없었어.
        

Question: 중학교때는 어땠어?
Answer:
        중학교때 선생님은 좀 싫었어.
        하지만 애들이 나를 많이 좋아했었어.
        그래서 연애도 많이 했지.
        

Question: 고등학교때는 어땠어?
Answer:
        고등학교때 선생님은 예뻤어.
        애들도 많이 좋아했었어. 
        그래서 나도 공부를 열심히 했지.
        

Question: 대학교때는 어땠어?


In [ ]:
# chatbot.invoke(few_shot_prompt.format(input="대학교때는 어땠어?"))

## Example selector

세상에 질문은 많고, 인간의 감정을 '배워야만 알 수 있는' AI한계는 분명하다. 다양한 유형의 요구에, 붙어 앉아서 적절한 example set을 먹여줘야 원하는 결과를 얻을 수 있다는 건 어불성설이니, 몇가지 예시덩어리를 던져주고, 사용자의 요구에 따라 적절한 예시덩어리를 '선택'하여 활용할 수 있도록 해야 한다.  

단, 이 '선택하는 행위'를 LLM이 직접 하게되면, 그 많은 예시덩어리를 다 일단 넘겨서(자원낭비, 리스크컨트롤) 죄다 한번씩 훑어보게 해야 할 것이다. 그래서 중간관리자가 살펴보고 적절한 예시덩어리를 골라서 최종처리자 LLM에게 던져줄 필요가 발생한다.

- 이게 느낌상... router같은데...
- agent tool 이랑 langgraph 같기도 하고?

In [ ]:
# !pip install -q chromadb tiktoken

### Completion

#### 1. 내가 원하는 건 이거야.

In [ ]:
# 최종적으로 내가 원하는 답변 템플릿 구성.
example_prompt = PromptTemplate(
    input_variables = ['input', 'output'],
    template='input: {input}\nOutput:{output}'
)

#### 2. 네가 사용할 수 있는 자료들

In [ ]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "정치인들에 대해서 어떻게 생각해?", "output": "정치이야기는 싫어"},
    {"input": "사내정치에 대해서 어떻게 생각해?", "output": "정치이야기는 싫어"},
    {"input": "종교인들에 대해서 어떻게 생각해?", "output": "종교이야기는 싫어"},
    {"input": "결혼생활에 대해서 어떻게 생각해?", "output": "결혼은 하면 미친짓, 안하면 바보짓이야"},
    {"input": "대학생활에 대해서 어떻게 생각해?", "output": "대학은 가면 자원낭비 안가면 인생낭비야"},
    {
        "input": "나비에 대한 노래를 불러줘",
        "output": "나비야 나비야 이리날아 오너라.",
    },
    {
        "input": "학교 종에 대한 노래를 불러줘",
        "output": "학교종이 떙떙땡 어서 모이자",
    },
]

#### 3. 너를 도와주실 매니저님을 소개할께.

In [ ]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [ ]:
to_vectorize = [" ".join(example.values()) for example in examples]
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [ ]:
my_manager = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
# my_manager.select_examples({"input": "병아리"})

'''
# 이렇게 통으로 해버리는 것도 가능은 하지만 좋은 예는 아닌듯?
# 계속 embedding을 반복할 테니.
 my_manager_total = SemanticSimilarityExampleSelector.from_examples(
     examples,
     OpenAIEmbeddings(),
     Chroma,
     k=1
 )
'''

'\n# 이렇게 통으로 해버리는 것도 가능은 하지만 좋은 예는 아닌듯?    \n# 계속 embedding을 반복할 테니.  \n my_manager_total = SemanticSimilarityExampleSelector.from_examples(\n     examples,\n     OpenAIEmbeddings(),\n     Chroma,\n     k=1\n )\n'

#### 4. 그러니까 말이지...

##### 매니저가 없는 경우

In [ ]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"],
)

print(prompt.format(input="병아리"))

input: 2+2
Output:4

input: 2+3
Output:5

input: 2+4
Output:6

input: 정치인들에 대해서 어떻게 생각해?
Output:정치이야기는 싫어

input: 사내정치에 대해서 어떻게 생각해?
Output:정치이야기는 싫어

input: 종교인들에 대해서 어떻게 생각해?
Output:종교이야기는 싫어

input: 결혼생활에 대해서 어떻게 생각해?
Output:결혼은 하면 미친짓, 안하면 바보짓이야

input: 대학생활에 대해서 어떻게 생각해?
Output:대학은 가면 자원낭비 안가면 인생낭비야

input: 나비에 대한 노래를 불러줘
Output:나비야 나비야 이리날아 오너라.

input: 학교 종에 대한 노래를 불러줘
Output:학교종이 떙떙땡 어서 모이자

Question: 병아리


##### 매니저가 있는 경우

In [ ]:
question = "병아리"
selected_examples = my_manager.select_examples({"question": question})
# print(f"Examples most similar to the input: {question}")
# for example in selected_examples:
#     print("\n")
#     for k, v in example.items():
#         print(f"{k}: {v}")


prompt_comp = FewShotPromptTemplate(
    example_selector = my_manager, # 매니저님이 골라주시는 샘플을 참고해서
    example_prompt = example_prompt, # 내가 처음에 물었던 대로 대답해주면 되.
    prefix='사용자의 질문유형에 맞는 대답을 해줘~', # 이건 optional.
    suffix='input:{input}', # 내 질문이 뭔지는 이따가 알려줄텐데
    input_variables =['input'] # input 이라는 자리에 넣어서 잘 생각해봐.
)

In [ ]:
ic(prompt_comp.format(input='병아리'))

ic| prompt_comp.format(input='병아리'): ('사용자의 질문유형에 맞는 대답을 해줘~
                                     '
                                      '
                                     '
                                      'input: 나비에 대한 노래를 불러줘
                                     '
                                      'Output:나비야 나비야 이리날아 오너라.
                                     '
                                      '
                                     '
                                      'input: 학교 종에 대한 노래를 불러줘
                                     '
                                      'Output:학교종이 떙떙땡 어서 모이자
                                     '
                                      '
                                     '
                                      'input:병아리')


'사용자의 질문유형에 맞는 대답을 해줘~\n\ninput: 나비에 대한 노래를 불러줘\nOutput:나비야 나비야 이리날아 오너라.\n\ninput: 학교 종에 대한 노래를 불러줘\nOutput:학교종이 떙떙땡 어서 모이자\n\ninput:병아리'

In [ ]:
compbot.invoke(prompt_comp.format(input='병아리'))

'에 대한 노래를 불러줘\nOutput:병아리병아리 꼬끼오 이리와 저리와 병아리야'

### Chat (아마도 여기서 RAG로 바로 이어지겠지)

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
)

#### completion과 동일한 부분.

In [ ]:
# 내가 원하는건 이거야
example_chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# 네가 사용할 수 있는 자료는 이거야
# examples
# 너를 도와주실 매니저님을 소개할께.
# my_manager

Define the few-shot prompt.
```python
selected_example = FewShotChatMessagePromptTemplate(
    # The input variables select the values to pass to the example_selector
    input_variables=["input"],
    example_selector=my_manager,
    # Define how each example will be formatted.
    # In this case, each example will become 2 messages:
    # 1 human, and 1 AI
    example_prompt=ChatPromptTemplate.from_messages(
        [("human", "{input}"), ("ai", "{output}")]
    ), # example_prompt를 따로 정의하지 않고 바로 여기서 넘기는 것도 가능.
)
```

##### 매니저가 없는 경우

In [ ]:
# 그래서 결론적으로 다시 정리하자면...
few_shot_chat = FewShotChatMessagePromptTemplate(
    example_prompt = example_chat_prompt, # 내가 처음에 물었던 대로 대답해주면 되.
    examples = examples, # 자료를 참고해서
    # prefix='', # 이건 optional.
    # suffix='Input:{input}\nOutput:', # 내 질문이 뭔지는 이따가 알려줄텐데
    input_variables =['input'] # input 이라는 자리에 넣어서 잘 생각해봐.
)

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자의 질문유형에 맞는 대답을 해줘~"),
        few_shot_chat,
        ("human", "{input}"),
    ]
)

In [ ]:
chatbot.invoke(final_prompt.format(input='병아리'))

AIMessage(content='AI: 병아리 병아리 삐약 삐약')

##### 매니저가 있는 경우

In [ ]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [ ]:
# 그래서 결론적으로 다시 정리하자면...
few_shot_chat_manager = FewShotChatMessagePromptTemplate(
    example_prompt = example_chat_prompt,
    example_selector=my_manager,
    # prefix='사용자의 질문유형에 맞는 대답을 해줘~', # 이건 optional. > 사용불가.
    # suffix='Input:{input}\nOutput:', # 내 질문이 뭔지는 이따가 알려줄텐데 > 사용불가
    input_variables =['input'] # input 이라는 자리에 넣어서 잘 생각해봐.
    # example_prompt=ChatPromptTemplate.from_messages(
    #     [("human", "{input}"), ("ai", "{output}")]
    # ),
)

In [ ]:
my_manager.select_examples({"input": "병아리"})

[{'input': '나비에 대한 노래를 불러줘', 'output': '나비야 나비야 이리날아 오너라.'},
 {'input': '학교 종에 대한 노래를 불러줘', 'output': '학교종이 떙떙땡 어서 모이자'}]

In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자의 질문유형에 맞는 대답을 해줘~"),
        few_shot_chat_manager,
        ("human", "{input}"),
    ]
)

In [ ]:
chatbot.invoke(final_prompt.format(input='병아리'))

AIMessage(content='AI: 병아리 병아리 꼬끼오, 작은 날개 펄럭이며 행복하게 살아요~')

여기서 final_prompt를 뜯어보면 말이지.. 이런 복잡한 모양이래.
```python
ChatPromptTemplate(
    input_variables=['input'],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=[],
                template='사용자의 질문유형에 맞는 대답을 해줘~'
            )
        ),
        FewShotChatMessagePromptTemplate(
            examples=[
                {'input': '2+2', 'output': '4'},
                {'input': '2+3', 'output': '5'},
                {'input': '2+4', 'output': '6'},
                {'input': '정치인들에 대해서 어떻게 생각해?', 'output': '정치이야기는 싫어'},
                {'input': '사내정치에 대해서 어떻게 생각해?', 'output': '정치이야기는 싫어'},
                {'input': '종교인들에 대해서 어떻게 생각해?', 'output': '종교이야기는 싫어'},
                {'input': '결혼생활에 대해서 어떻게 생각해?', 'output': '결혼은 하면 미친짓, 안하면 바보짓이야'},
                {'input': '대학생활에 대해서 어떻게 생각해?', 'output': '대학은 가면 자원낭비 안가면 인생낭비야'},
                {'input': '나비에 대한 노래를 불러줘', 'output': '나비야 나비야 이리날아 오너라.'},
                {'input': '학교 종에 대한 노래를 불러줘', 'output': '학교종이 떙떙땡 어서 모이자'}
            ],
            input_variables=['input'],
            example_prompt=ChatPromptTemplate(
                input_variables=['input', 'output'],
                messages=[
                    HumanMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['input'],
                            template='{input}'
                        )
                    ),
                    AIMessagePromptTemplate(
                        prompt=PromptTemplate(
                            input_variables=['output'],
                            template='{output}'
                        )
                    )
                ]
            )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['input'],
                template='{input}'
            )
        )
    ]
)
```